# Planner: explanation and examples

This explains the bitplanning code and setup.

You must define a **Domain**, which is a set of states and actions.  This is like a "world".  

There may be variables in the actions in a Domain, but to solve the problems you must create a concrete world by enumerating all the values for the variables. This creates a ConcreteDomain, and explodes out the actions and states.

In [77]:
from parsedomain import Domain

## Defining a domain

A domain contains **states** and **actions**.  Each state may be true or false or unknown/unspecified/uninteresting.

A domain is defined with a big string that uses a particular syntax.  It's naive and line-based (though not indentation-aware), so the vertical nature of what you see is required.  Keywords must be on a line of their own.

States are defined like:

```
state
    StateName(v1, v2)
where [optional]
    v1 is some_variable
    v2 is some_variable
    v1 != v2
```

That is, a line with `state` on it, the name of a state (possibly with variables), and on the next line the name of the state.  Names are opaque, and besides leading and trailing whitespace they are whitespace and case sensitive. I.e., `StateName(v1, v2)` and `StateName(v1,v2)` wouldn't match.

The `where` clause is available for most commands. It introduces substitutions. There are two kinds of statements: `v1 is some_variable` which says that the string `v1` will be substituted for all values of `some_variable`.  In this case it would be all combinations of `v1` and `v2` (i.e., if `some_variables` had 10 values, there would be 100 states defined). You can also restrict using `v1 != v2`.

To define an action, you use something similar:

```
action
    Move(o, p)
must
    OnTable(o)
    Clear(p)
then
    On(o, p)
    not Clear(p)
where
    o is object
    p is place
```

Note the clauses can be in any order.  `must` is all the requirements for the action to be invoked, and `then` is the result of the actions. These each must be states.  Note you can add `not` before any action to assert the state must or will be false.

The last kind of definition is a **constraint**.  These are optional, but help fill in details later. These are the implications of a state.  For instance, if an object is in one place, it can't be in other places:

```
if
    On(o, p)
then
    not On(o, p2)
    not OnTable(o)
where
    o is object
    p is place
    p2 is place
    p != p2
```

This might catch errors elsewhere, and also makes it easier to setup initial state, e.g., if you say `On(A, Location1)` then the system can infer that `not On(A, Location2)`.

In [78]:
cargo_domain = Domain("cargo_domain", """
# This is a domain where cargo and planes can be At(thing, airport), or a piece of cargo can be In(cargo, airplane)
# Planes can fly to any other airport. Cargo can be loaded into a plane like Load(cargo, plane, location), or 
# unloaded like Unload(cargo, plane, location).
state
    At(p, a)
where
    p is plane
    a is airport

state
    At(c, a)
where
    c is cargo
    a is airport

state
    In(c, p)
where
    c is cargo
    p is plane

if
    At(p, a)
then
    not At(p, a2)
where
    p is plane
    a is airport
    a2 is airport
    a != a2

if
    At(c, a)
then
    not At(c, a2)
    not In(c, p)
where
    c is cargo
    a is airport
    a2 is airport
    a != a2
    p is plane

if
    In(c, p)
then
    not At(c, a)
    not In(c, p2)
where
    c is cargo
    a is airport
    p is plane
    p2 is plane
    p != p2

to
    Fly(p, a1, a2)
where
    p is plane
    a1 is airport
    a2 is airport
    a1 != a2
must
    At(p, a1)
then
    At(p, a2)
    not At(p, a1)

to
    Load(c, p, a)
where
    c is cargo
    p is plane
    a is airport
must
    At(c, a)
    At(p, a)
then
    In(c, p)
    not At(c, a)

to
    Unload(c, p, a)
where
    c is cargo
    p is plane
    a is airport
must
    In(c, p)
    At(p, a)
then
    At(c, a)
    not In(c, p)
""")

## Binding variables in a domain

In the previous examples we had unbound variables (e.g., `cargo`). To actually solve problems these must be enumerated, and many states and actions will be created.

Some problems don't have variables, but even then you must bind variables (there's examples of this later), like `a_domain.substitute({})`.

You can bind variables like:

```
simple_cargo = cargo_domain.substitute({"cargo": ["C1", "C2], ...})
```

Or you can use a single string like below:

In [79]:
simple_cargo = cargo_domain.substitute("""
cargo: C1 C2
plane: P1 P2
airport: JFK SFO
""")

In [80]:
# Note that the substituted domain has a nice representation (you could also print(simple_cargo)):
simple_cargo

State name,BitMask
"At(C1, JFK)",A-----------
"At(C1, SFO)",-B----------
"At(C2, JFK)",--C---------
"At(C2, SFO)",---D--------
"At(P1, JFK)",----E-------
"At(P1, SFO)",-----F------
"At(P2, JFK)",------G-----
"At(P2, SFO)",-------H----
"In(C1, P1)",--------I---
"In(C1, P2)",---------J--


## Defining problems

A problem is a ConcreteDomain with an initial state (`start`) and a `goal`.

The start is defined by a string with one state on each line. You can assert that everything not specified is false using `default_false` on a line by its own, or if you define constraints it may be able to determine all the consequences of your assertions. If not everything is specified then you'll get an error.

The goal is defined similarly, though of course it doesn't have to be fully specified.

In [81]:
simple_cargo_problem = simple_cargo.problem(
start="""
At(C1, SFO)
At(C2, SFO)
At(P1, SFO)
At(P2, JFK)
""",
goal="""
At(C2, JFK)
At(C1, JFK)
""")

## Solving problems

Once you've defined a problem, solving it is as simple as `problem.solve()`. This returns the solution as an ActionSequence, or None if no solution can be found.

In [82]:
simple_cargo_problem.solve()

<ActionSequence Load(C1, P1, SFO) Load(C2, P1, SFO): Fly(P1, SFO, JFK): Unload(C1, P1, JFK) Unload(C2, P1, JFK): >

### Understanding how the solution is found

Along the way to the solution, we try lots of things. It's interesting to watch!  The problem saves a `.log`, which you can print or watch in the notebook.

Note that everything shows BitMasks.  These are fields of true/false/doesn't-matter. Each state is one item. It's equivalent to a binary number, but we give each digit its own letter so it's easier to follow.  A string like `A-C---------` means that A and C are true, and the rest don't matter (`a` means that item is false).  Each BitMask is actually two binary numbers: the true/false set, and the matters/doesn't-matter set.

In [83]:
simple_cargo_problem.log

## All about the solver

The solver works backwards from the goal.

The basic theory:
    
* The solution is false until it is true. Therefore the last action must be something that makes the goal true, and doesn't undo any part of the goal.
* We can think about sets of actions instead of individual actions. These sets of actions (called ActionPool) can all co-exist:
  * No action can have an effect that invalidates the prerequisite of another action in the pool
  * No action can have a prerequisite that conflicts with the prerequisite of another action in the pool
  * No action can have an effect that is in conflict with the prerequisite of another action in the pool
* When we pick a "last" action, then we have a new goal, which is the requirements of the last action, and any goal requirements that the last action didn't satisfy.
* There are many possible "last" actions, so we create a set of options (called the "frontier" in the code). We pick what we consider the "best" option, using east option's "score" (this is done in `Domain.score_accomplishment_pool()`).
* When we pick the best-looking option, we remove it from the frontier, we test whether it is a solution, and if not then we consider actions that could happen before it which could make it part of a full solution. All these options are scored and added to the frontier, and may be picked later.
* If some sequence of actions has a prerequisite (`must`) that is identical to something else we've seen, then we throw it away. This avoids loops.

# A bunch more examples

Below are a bunch of examples. `fixit` is the one we can't solve, *sadface*

In [84]:
air_cargo_p1 = cargo_domain.substitute("""
cargo: C1 C2
plane: P1 P2
airport: JFK SFO
""")
air_cargo_p1

State name,BitMask
"At(C1, JFK)",A-----------
"At(C1, SFO)",-B----------
"At(C2, JFK)",--C---------
"At(C2, SFO)",---D--------
"At(P1, JFK)",----E-------
"At(P1, SFO)",-----F------
"At(P2, JFK)",------G-----
"At(P2, SFO)",-------H----
"In(C1, P1)",--------I---
"In(C1, P2)",---------J--


In [85]:
air_cargo_p1_problem = air_cargo_p1.problem(
start="""
At(C1, SFO)
At(C2, JFK)
At(P1, SFO)
At(P2, JFK)
""",
goal="""
At(C1, JFK)
At(C2, SFO)
""")

In [86]:
air_cargo_p1_problem.solve()

<ActionSequence Fly(P2, JFK, SFO): Load(C1, P2, SFO): Fly(P1, SFO, JFK) Fly(P2, SFO, JFK): Load(C2, P1, JFK): Fly(P1, JFK, SFO): Unload(C1, P2, JFK) Unload(C2, P1, SFO): >

In [87]:
air_cargo_p1_problem.log

In [88]:
air_cargo_p2_problem = cargo_domain.substitute("""
cargo: C1 C2 C3
plane: P1 P2 P3
airport: JFK SFO ATL
""").problem(
start="""
At(C1, SFO)
At(C2, JFK)
At(C3, ATL)
At(P1, SFO)
At(P2, JFK)
At(P3, ATL)
""",
goal="""
At(C1, JFK)
At(C2, SFO)
At(C3, SFO)
""")
air_cargo_p2_problem.solve()

<ActionSequence Fly(P3, ATL, JFK): Load(C2, P3, JFK): Fly(P2, JFK, SFO) Fly(P3, JFK, ATL): Load(C1, P2, SFO): Fly(P2, SFO, JFK) Load(C3, P3, ATL): Fly(P3, ATL, SFO): Unload(C1, P2, JFK) Unload(C2, P3, SFO) Unload(C3, P3, SFO): >

In [89]:
air_cargo_p2_problem.log

In [90]:
air_cargo_p3_problem = cargo_domain.substitute("""
cargo: C1 C2 C3 C4
plane: P1 P2
airport: JFK SFO ATL ORD
""").problem(
start="""
At(C1, SFO)
At(C2, JFK)
At(C3, ATL)
At(C4, ORD)
At(P1, SFO)
At(P2, JFK)
""",
goal="""
At(C1, JFK)
At(C2, SFO)
At(C3, JFK)
At(C4, SFO)
""")
air_cargo_p3_problem.solve()

<ActionSequence Fly(P2, JFK, SFO): Load(C1, P2, SFO): Fly(P2, SFO, ATL): Load(C3, P2, ATL): Fly(P1, SFO, ORD) Fly(P2, ATL, JFK): Load(C4, P1, ORD): Fly(P1, ORD, JFK): Load(C2, P1, JFK): Fly(P1, JFK, SFO): Unload(C1, P2, JFK) Unload(C2, P1, SFO) Unload(C3, P2, JFK) Unload(C4, P1, SFO): >

In [91]:
have_cake_problem = Domain("have_cake", """
state
  HaveCake
state
  EatenCake

to
  Eat
must
  HaveCake
then
  not HaveCake
  EatenCake

to
  MakeCake
then
  HaveCake
""").substitute({}).problem(
start="""
HaveCake
not EatenCake
""",
goal="""
HaveCake
EatenCake
""")
have_cake_problem.solve()

<ActionSequence Eat: MakeCake: >

In [92]:
spare_tire = Domain("spare_tire", """
state
    At(t, p)
where
    t is tire
    p is place

to
    Remove(t, r)
where
    t is tire
    r is removable
must
    At(t, r)
then
    At(t, Ground)
    
to
    PutOn(t, Axle)
where
    t is tire
    t2 is tire
    t != t2
must
    At(t, Ground)
    not At(t2, Axle)
then
    At(t, Axle)
    not At(t, Ground)
    
to
    LeaveOvernight
then
    not At(Spare, Ground)
    not At(Spare, Trunk)
    not At(Spare, Axle)
    not At(Flat, Ground)
    not At(Flat, Trunk)
    not At(Flat, Axle)
    
if
    At(t, p)
then
    not At(t, p2)
where
    t is tire
    p is place
    p2 is place
    p != p2
""").substitute("""
tire: Flat Spare
place: Axle Ground Trunk
removable: Axle Trunk
""")
spare_tire

State name,BitMask
"At(Flat, Axle)",A-----
"At(Flat, Ground)",-B----
"At(Flat, Trunk)",--C---
"At(Spare, Axle)",---D--
"At(Spare, Ground)",----E-
"At(Spare, Trunk)",-----F


In [93]:
spare_tire_problem = spare_tire.problem(
start="""
At(Flat, Axle)
At(Spare, Trunk)
""",
goal="""
At(Spare, Axle)
""")
spare_tire_problem.solve()

<ActionSequence Remove(Flat, Axle) Remove(Spare, Trunk): PutOn(Spare, Axle): >

In [94]:
spare_tire_problem.log

In [95]:
dinner_date = Domain("dinner_date", """
state
    GarbageInside
state
    HandsClean
state
    IsQuiet
state
    DinnerCooked
state
    PresentWrapped
    
to
    Cook
must
    HandsClean
then
    DinnerCooked
    
to
    Wrap
must
    IsQuiet
then
    PresentWrapped
    
to
    CarryOutGarbage
then
    not GarbageInside
    not HandsClean
    
to
    DollyOutGarbage
then
    not GarbageInside
    not IsQuiet
""").substitute({})
dinner_date

State name,BitMask
DinnerCooked,A----
GarbageInside,-B---
HandsClean,--C--
IsQuiet,---D-
PresentWrapped,----E


In [96]:
dinner_date_problem = dinner_date.problem(
start="""
GarbageInside
HandsClean
IsQuiet
not DinnerCooked
not PresentWrapped
""",
goal="""
DinnerCooked
PresentWrapped
not GarbageInside
""")
dinner_date_problem.solve()

<ActionSequence Cook: CarryOutGarbage Wrap: >

In [97]:
dinner_date_problem.log

# Blocks

From the blocks descriptions in [this graphplan directory](http://www.cs.cmu.edu/afs/cs.cmu.edu/usr/avrim/Planning/Graphplan/)

In [98]:
block_domain = Domain("block_domain", """
state
    On(o, under)
where
    o is object
    under is object
    o != under

state
    OnTable(o)
where
    o is object

state
    Clear(o)
where
    o is object

state
    Holding(o)
where
    o is object

state
    ArmEmpty

to
    PickUp(o)
where
    o is object
must
    Clear(o)
    OnTable(o)
    ArmEmpty
then
    Holding(o)
    not ArmEmpty

to
    PutDown(o)
where
    o is object
must
    Holding(o)
then
    Clear(o)
    not Holding(o)
    ArmEmpty
    OnTable(o)

to
    Stack(o, under)
where
    o is object
    under is object
    o != under
must
    Clear(under)
    Holding(o)
then
    not Holding(o)
    ArmEmpty
    Clear(o)
    On(o, under)

to
    Unstack(o, under)
where
    o is object
    under is object
    o != under
must
    On(o, under)
    Clear(o)
    ArmEmpty
then
    Holding(o)
    not ArmEmpty
    Clear(under)

if
    On(o, under)
where
    o is object
    under is object
    o != under
    other is object
    other != under
    other != o
then
    not Clear(under)
    not OnTable(o)
    not Holding(o)
    not Holding(under)
    not On(o, other)
    
if
    OnTable(o)
where
    o is object
    under is object
    o != under
then
    not Holding(o)
    not On(o, under)

if
    Holding(o)
where
    o is object
    under is object
    o != under
then
    not ArmEmpty
    not On(o, under)
    not OnTable(o)
    
if
    ArmEmpty
where
    o is object
then
    not Holding(o)
    
if
    Clear(o)
where
    o is object
    over is object
    o != over
then
    not On(over, o)
""")

In [99]:
block_suss = block_domain.substitute("""object: A B C""").problem(
start="""
OnTable(A)
OnTable(B)
On(C, A)
Clear(B)
Clear(C)
ArmEmpty
""",
goal="""
On(A, B)
On(B, C)
""")
block_suss.solve()

<ActionSequence Unstack(C, A): PutDown(C): PickUp(B): Stack(B, C): PickUp(A): Stack(A, B): >

In [100]:
blocks_4 = block_domain.problem(
bindings="""
object: A B C D
""",
start="""
OnTable(A)
On(B, A)
On(C, B)
On(D, C)
Clear(D)
ArmEmpty
""",
goal="""
On(B, A)
On(C, B)
On(A, D)
""")
blocks_4.solve()

<ActionSequence Unstack(D, C): PutDown(D): Unstack(C, B): PutDown(C): Unstack(B, A): Stack(B, C): PickUp(A): Stack(A, D): Unstack(B, C): Stack(B, A): PickUp(C): Stack(C, B): >

In [101]:
blocks_5 = block_domain.problem(
bindings="""
object: A B C D E
""",
start="""
OnTable(A)
On(B, A)
On(C, B)
On(D, C)
On(E, D)
Clear(E)
ArmEmpty
""",
goal="""
On(B, A)
On(C, B)
On(D, C)
On(A, E)
""")
blocks_5.solve()

<ActionSequence Unstack(E, D): PutDown(E): Unstack(D, C): PutDown(D): Unstack(C, B): Stack(C, D): Unstack(B, A): Stack(B, C): PickUp(A): Stack(A, E): Unstack(B, C): Stack(B, A): Unstack(C, D): Stack(C, B): PickUp(D): Stack(D, C): >

In [102]:
blocks_impossible = block_domain.problem(
bindings="""
object: A B C
""",
start="""
OnTable(A)
On(B, A)
On(C, B)
Clear(C)
ArmEmpty
""",
goal="""
On(C, B)
On(B, A)
On(A, C)
""")
blocks_impossible.solve()

In [103]:
# The log looks very minimal, but we can detect that the solution is impossible right away because
# there's no "last" action that solves a goal state and doesn't invalidate a goal state.
blocks_impossible.log

In [104]:
tsp_domain = Domain("tsp", """
state
    At(l)
where
    l is location
    
state
    Visited(l)
where
    l is location
    
state
    (l1 l2 CONNECTED)
where
    l1 is location
    l2 is location
    l1 != l2
    
to
    Move(start, end)
where
    start is location
    end is location
    start != end
must
    At(start)
    (start end CONNECTED)
then
    At(end)
    Visited(end)

if
    At(l)
where
    l is location
    l2 is location
    l != l2
then
    not At(l2)
""")

In [105]:
tsp_world = tsp_domain.problem(
bindings="""
location: A B C D E F G H I
""",
start="""
default_false

(A B CONNECTED)
(A C CONNECTED)
(A D CONNECTED)
(A E CONNECTED)
(A F CONNECTED)
(A G CONNECTED)
(A H CONNECTED)
(A I CONNECTED)

(B A CONNECTED)
(B C CONNECTED)
(B D CONNECTED)
(B E CONNECTED)
(B F CONNECTED)
(B G CONNECTED)
(B H CONNECTED)
(B I CONNECTED)

(C A CONNECTED)
(C B CONNECTED)
(C D CONNECTED)
(C E CONNECTED)
(C F CONNECTED)
(C G CONNECTED)
(C H CONNECTED)
(C I CONNECTED)

(D A CONNECTED)
(D B CONNECTED)
(D C CONNECTED)
(D E CONNECTED)
(D F CONNECTED)
(D G CONNECTED)
(D H CONNECTED)
(D I CONNECTED)

(E A CONNECTED)
(E B CONNECTED)
(E C CONNECTED)
(E D CONNECTED)
(E F CONNECTED)
(E G CONNECTED)
(E H CONNECTED)
(E I CONNECTED)

(F A CONNECTED)
(F B CONNECTED)
(F C CONNECTED)
(F D CONNECTED)
(F E CONNECTED)
(F G CONNECTED)
(F H CONNECTED)
(F I CONNECTED)

(G A CONNECTED)
(G B CONNECTED)
(G C CONNECTED)
(G D CONNECTED)
(G E CONNECTED)
(G F CONNECTED)
(G H CONNECTED)
(G I CONNECTED)

(H A CONNECTED)
(H B CONNECTED)
(H C CONNECTED)
(H D CONNECTED)
(H E CONNECTED)
(H F CONNECTED)
(H G CONNECTED)
(H I CONNECTED)

(I A CONNECTED)
(I B CONNECTED)
(I C CONNECTED)
(I D CONNECTED)
(I E CONNECTED)
(I F CONNECTED)
(I G CONNECTED)
(I H CONNECTED)

At(A)

""",
goal="""
At(A)
Visited(A)
Visited(B)
Visited(C)
Visited(D)
Visited(E)
Visited(F)
Visited(G)
Visited(H)
Visited(I)
""")
tsp_world.solve()

<ActionSequence Move(A, I): Move(I, H): Move(H, G): Move(G, F): Move(F, E): Move(E, D): Move(D, C): Move(C, B): Move(B, A): >

In [106]:
tsp_world.log

In [107]:
tsp_peterson = tsp_domain.problem(
bindings="""
location: A B C D E F G H I J
""",
start="""
default_false
(A B CONNECTED)
(B A CONNECTED)
(A C CONNECTED)
(C A CONNECTED)
(A I CONNECTED)
(I A CONNECTED)
(B F CONNECTED)
(F B CONNECTED)
(B H CONNECTED)
(H B CONNECTED)
(C D CONNECTED)
(D C CONNECTED)
(C E CONNECTED)
(E C CONNECTED)
(D H CONNECTED)
(H D CONNECTED)
(D J CONNECTED)
(J D CONNECTED)
(E F CONNECTED)
(F E CONNECTED)
(E G CONNECTED)
(G E CONNECTED)
(F J CONNECTED)
(J F CONNECTED)
(G H CONNECTED)
(H G CONNECTED)
(G I CONNECTED)
(I G CONNECTED)
(I J CONNECTED)
(J I CONNECTED)

At(A)
""",
goal="""
At(A)
Visited(A)
Visited(B)
Visited(C)
Visited(D)
Visited(E)
Visited(F)
Visited(G)
Visited(H)
Visited(I)
Visited(J)
""")
tsp_peterson.solve()

<ActionSequence Move(A, I): Move(I, J): Move(J, I): Move(I, G): Move(G, H): Move(H, D): Move(D, C): Move(C, E): Move(E, F): Move(F, B): Move(B, A): >

In [108]:
tsp_facts = tsp_domain.problem(
bindings="""
location: Boston NewYork Pittsburgh Toronto Albany
""",
start="""
default_false
(Boston NewYork CONNECTED)
(NewYork Boston CONNECTED)
(Pittsburgh Boston CONNECTED)
(Boston Pittsburgh CONNECTED)
(Pittsburgh NewYork CONNECTED)
(NewYork Pittsburgh CONNECTED)
(Toronto Pittsburgh CONNECTED)
(Toronto NewYork CONNECTED)
(NewYork Toronto CONNECTED)
(NewYork Albany CONNECTED)
(Albany NewYork CONNECTED)
(Albany Toronto CONNECTED)
(Toronto Albany CONNECTED)
At(Pittsburgh)
""",
goal="""
Visited(Boston)
Visited(NewYork)
Visited(Pittsburgh)
Visited(Toronto)
Visited(Albany)
At(Pittsburgh)
""")
tsp_facts.solve()

<ActionSequence Move(Pittsburgh, NewYork): Move(NewYork, Toronto): Move(Toronto, Albany): Move(Albany, NewYork): Move(NewYork, Boston): Move(Boston, Pittsburgh): >

## Fixit, our nemesis

This example (from [here](http://www.cs.cmu.edu/afs/cs.cmu.edu/usr/avrim/Planning/Graphplan/)) is hard, and the planner makes very little progress on it.  `python fixit_exampe.py` also runs this. Because it doesn't come to a solution it can be easier to run it from the command-line.

In [109]:
fixit_domain = Domain("fixit", """
to
    (open c)
where
    c is container
must
    (unlocked c)
    (closed c)
then
    not (closed c)
    (open c)
    
to
    (close c)
where
    c is container
must
    (open c)
then
    not (open c)
    (closed c)

to
    (fetch o c)
where
    o is object
    c is container
must
    (in o c)
    (open c)
then
    not (in o c)
    (have o)

to
    (put-away o c)
where
    o is object
    c is container
must
    (have o)
    (open c)
then
    (in o c)
    not (have o)

to
    (loosen n h)
where
    n is nut
    h is hub
must
    (have wrench)
    (tight n h)
    (on-ground h)
then
    (loose n h)
    not (tight n h)

to
    (tighten n h)
where
    n is nut
    h is hub
must
    (have wrench)
    (loose n h)
    (on-ground h)
then
    (tight n h)
    not (loose n h)

to
    (jack-up h)
where
    h is hub
must
    (on-ground h)
    (have jack)
then
    (not-on-ground h)
    not (on-ground h)
    not (have jack)

to
    (jack-down h)
where
    h is hub
must
    (not-on-ground h)
then
    not (not-on-ground h)
    (on-ground h)
    (have jack)

to
    (undo n h)
where
    n is nut
    h is hub
must
    (not-on-ground h)
    (fastened h)
    (have wrench)
    (loose n h)
then
    (have n)
    (unfastened h)
    not (fastened h)
    not (loose n h)

to
    (do-up n h)
where
    n is nut
    h is hub
must
    (have wrench)
    (unfastened h)
    (not-on-ground h)
    (have n)
then
    (loose n h)
    (fastened h)
    not (unfastened h)
    not (have n)

to
    (remove-wheel w h)
where
    w is wheel
    h is hub
must
    (not-on-ground h)
    (on w h)
    (unfastened h)
then
    (have w)
    (free h)
    not (on w h)

to
    (put-on-wheel w h)
where
    w is wheel
    h is hub
must
    (have w)
    (free h)
    (unfastened h)
    (not-on-ground h)
then
    (on w h)
    not (free h)
    not (have w)
    
to
    (inflate w)
where
    w is wheel
must
    (have pump)
    (not-inflated w)
    (intact w)
then
    not (not-inflated w)
    (inflated w)

to
    cuss
then
    not annoyed
""")

In [110]:
fixit_1 = fixit_domain.substitute("""
object: wrench jack pump the-hub nuts wheel1 wheel2
hub: the-hub
nut: nuts
container: boot
wheel: wheel1 wheel2
""")
fixit_1

State name,BitMask
(closed boot),A--------------------------------
(fastened the-hub),-B-------------------------------
(free the-hub),--C------------------------------
(have jack),---D-----------------------------
(have nuts),----E----------------------------
(have pump),-----F---------------------------
(have the-hub),------G--------------------------
(have wheel1),-------H-------------------------
(have wheel2),--------I------------------------
(have wrench),---------J-----------------------


In [111]:
fixit_1_problem = fixit_1.problem(
start="""
default_false
(intact wheel2) 
(in jack boot)
(in pump boot) 
(in wheel2 boot) 
(in wrench boot)
(on wheel1 the-hub)
(on-ground the-hub)
(tight nuts the-hub)
(not-inflated wheel2)
(unlocked boot)
(fastened the-hub)
(closed boot)
""",
goal="""
(on wheel2 the-hub)
(in wheel1 boot)
(inflated wheel2)
(in wrench boot)
(in jack boot)
(in pump boot)
(tight nuts the-hub)
(closed boot)
""")

In [112]:
# This doesn't work :(
# fixit_1_problem.solve()
# Should be something like:
# (open boot) 
# (fetch jack boot) (fetch wrench boot) (fetch wheel2 boot)
# (loosen nuts the-hub) (inflate wheel2)
# (jack-up the-hub)
# (undo nuts the-hub)
# (remove-wheel wheel1 the-hub)
# (put-away wheel1 the-hub) (put-on-wheel wheel2 the-hub)
# (put-on nuts the-hub)
# (jack-down the-hub)
# (tighten nuts the-hub)
# (put-away jack boot) (put-away wrench)
# (close boot)